# Write an IP problem in <'lp'> format - deer habitat model 

In [1]:
%load_ext autoreload
%autoreload 2

from pulp import * 
import pandas as pd 
import numpy as np 
import math 

In [2]:
#Import files 
## Read excel files with panda 
#! pip install xlrd 
#!  pip install docplex
#conda install -c ibmdecisionoptimization docplex
#conda update -c ibmdecisionoptimization docplex
df_sheet_3 = pd.read_excel('IP_PB_simpledeerhab.xlsx', sheet_name=1)
#df_sheet_3 = df_sheet_3.iloc[:10]

In [3]:
### TO DO 
##### TASKS ##### 
## initial class age ## 

## (WK_1: 09/05-09/10)  solve the optimization problem for an area of X stands (TB - 91 stands)
#### Done - Deer1 --> optimal 

##### Observations: 
## PCT ### most treatments arrive at the end (no i class age)
## All treatments at the same period 

## (WK_2) ensure that deer habitat would be maintained along the planning horizon.  
#### Option 1 
##### Could set an arbitrary threshold ~ Deer1 model OV : 12108 ac 
# ==> divide by the time period here 14 
# constraint ensures Sum a m*alpha m,p for all p >= OV/14 
# would it be good to add slack etc for sensitivity analysis? 

#### Option 2 
#### Overall buget splitted along 14 years 
#### PCT budget - just cost 
###### ANDREW 
#### 2017 cost of PCT/ac = $420/ac

#### harvest budget - cost? - LEV? - etc. close to road network
### find some proxi - maybe scheduling models for harvest 

#### Option 3
### define a constraint limiting the number of treatment per periods 
    ### Sum of PCT over all stands <= ai 
    ### Sum of harvest over all stands <= bi 
### run some sensitivity analysis
### number of actions or ac to cut per year*5 

#### (WK_3) Exclusivity problem 
### carying capacity 

#### OPTION 1: 
#### if cover is selected then sum of adjacent aphas*area (available forage) <= area cover 
###### since a maximization problem don't need a constraint that would force alpha so alpha can be tuned of 

#### OPTION 2: 
# Var Co 1 if cover selected 0 otherwise 
#I could set a shelter capacity to each old growth parcel and 
#whenever the forage quantity reached the shelter capacity requirements it would stop the 
#forage allocation (minimize surplus) …  

#WK_4 Wrap up:
#Do some sensibility analysis and assess 
#how many deer could stay on the area. 
#Track harvest events and estimate the volume of wood that we could collect.
#Present some maps of treatments allocations. 
#Compare results from “Max total” and “Max each period”. 
#Prepare for further research: What to do next? Ex: climate change considerations where snow 
#can kill 33% of fawns (is said to be most critical factor compared to predation), 
#hunt success indicator, forets productivity, wildlife corridor concerns …  

#### To do #1 
# add all the reference used for this model 

#### To do #2
# look for age-classes 

#### To do #3 
# do we need to set a constraint on harvest actions around 55 years? 
### COULD Set a contraint for Y m,t 
### Assume that forage is available 15 years after harvest 
### Assume : did not consider that PCT would transform faster the cover availability 

df_sheet_3 

,Stand_IPid,SSIZEC,Harvest,PCT_YR,Acres,Neighbor,Total_Area
0,1,_,1972,2003,71.291082,9_12,45.000000
1,10,_,1972,0,198.287017,21_24_89_8,611.861051
2,11,OG,0,0,0.950000,58_40,54.756393
3,12,OG,0,0,5.000000,1_9,111.291082
4,13,OG,0,0,28.000000,66_5,46.460384
...,...,...,...,...,...,...,...
86,88,_,1997,0,58.442032,77_50,259.551252
87,89,_,1972,0,198.287017,24_10_75,429.086712
88,9,OG,0,0,40.000000,15_1_12,147.582165
89,90,_,1962,0,21.356563,80_22_32_17_18_19_62_65,710.758524


In [4]:
# Data definition 

mid_point = 2.5 # if PCT or harvest, estimated to happen at mid-point of 5 year periods 
n_periods = 14 # if 70 years horizon, and 5-year periods then 70/5 = 14 
n_stands = len(df_sheet_3)
i_yr = 2020

##PCT
min_yr = 15
max_yr = 40

# Cover 
min_yr_cover = 120


M = n_stands + 1  
N_stands = df_sheet_3["Stand_IPid"]

Yr_harvest = df_sheet_3["Harvest"]
Yr_PCT = df_sheet_3["PCT_YR"]
Yr_OG = df_sheet_3["SSIZEC"]

A_stands = pd.DataFrame()
A_stands = A_stands.assign(area = df_sheet_3["Acres"])

#Adj_stands[["stand_i","stand_j"]] = df_sheet_3[["Stand_IPid", "Neigbor"]]
Adj_stands = pd.DataFrame()
Adj_stands = Adj_stands.assign(stand_i=df_sheet_3["Stand_IPid"], adj_i=df_sheet_3["Neighbor"])
Adj = Adj_stands["adj_i"].values.tolist()
i_Adj = Adj_stands["stand_i"].values.tolist()
i_Adj.sort()


#Adj_stands["stand_i"][1] or "adj_i"
Area_adjacent = df_sheet_3["Total_Area"]
Total_area = Area_adjacent+A_stands #directly adjacent cells area


In [ ]:
#df_sheet_3


In [5]:
# # Model initialization
model= LpProblem("Deer1", LpMaximize)


In [6]:
# Decision variables 
## X m, t - stand m to be PCTed in time period t  _ Xm,t=1; 0 otherwise 
X_variable_names = [str(i) + ',' + str(j) for i in range (1, n_stands+1) for j in range (n_periods+1)]

DV_X_variables = LpVariable.matrix("X", X_variable_names, cat = "Binary", lowBound= 0 )
X_allocation = np.array(DV_X_variables).reshape(len(df_sheet_3),14+1)

## Y m, t - stand m to be harvested in time period t _ Ym,t=1; 0 otherwise 
Y_variable_names = [str(i) + ',' + str(j) for i in range (1, n_stands+1) for j in range (n_periods+1)]

DV_Y_variables = LpVariable.matrix("Y", Y_variable_names, cat = "Binary", lowBound= 0 )
Y_allocation = np.array(DV_Y_variables).reshape(len(df_sheet_3),14+1)

### generate variables for previous PCT - RECORD X m,-t : 
add_var = []
new_X_m = []
new_X_p = []
for i in range (len(Yr_PCT)): 
    if Yr_PCT[i] != 0: 
        new_X_m.append(N_stands[i])
        new_X_p.append(math.floor((Yr_PCT[i]-i_yr-1)/5))
record_X_variables = [str(new_X_m[j]) + ',' + str(new_X_p[j]) for j in range (len(new_X_p))]

DV_Xr_variables = LpVariable.matrix("X", record_X_variables, cat = "Binary", lowBound= 0 )
Xr_allocation = np.array(DV_Xr_variables).reshape(len(new_X_p))
### generate variables for previous PCT - RECORD X m,-t :         
new_Y_m = []
new_Y_p = []
for i in range (len(Yr_harvest)): 
    if Yr_harvest[i] != 0: 
        new_Y_m.append(N_stands[i])
        new_Y_p.append(math.floor((Yr_harvest[i]-i_yr-1)/5))
    elif Yr_OG[i] == "OG":
        new_Y_m.append(N_stands[i])
        new_Y_p.append(-26) #considered min age for old growth (period of 5 years so 5*26)
        
record_Y_variables = [str(new_Y_m[j]) + ',' + str(new_Y_p[j]) for j in range (len(new_Y_p))]
DV_Yr_variables = LpVariable.matrix("Y", record_Y_variables, cat = "Binary", lowBound= 0 )
Yr_allocation = np.array(DV_Yr_variables).reshape(len(new_Y_p))

# Indicator variable 
## delta m,t - stand m to be considered as forage stand in time t _ delta m,t =1; 0 otherwise 
### generate for all stands 
#"\zeta[tab]"
#delta
delta_variable_names = [str(i) + ',' + str(j) for i in range (1, n_stands+1) for j in range (n_periods+1)]
IV_delta_variables = LpVariable.matrix("delta", delta_variable_names, cat = "Binary", lowBound= 0 )
delta_allocation = np.array(IV_delta_variables).reshape(len(df_sheet_3),14+1)
## omega m,t - cover habitat
#omega
omega_variable_names = [str(i) + ',' + str(j) for i in range (1, n_stands+1) for j in range (n_periods+1)]
IV_omega_variables = LpVariable.matrix("omega", omega_variable_names, cat = "Binary", lowBound= 0 )
omega_allocation = np.array(IV_omega_variables).reshape(len(df_sheet_3),14+1)
### find later how to replace string elements  

## alpha m,t - usable habitat by deer 
### generate for all stands 
#alpha
alpha_variable_names = [str(i) + ',' + str(j) for i in range (1, n_stands+1) for j in range (n_periods+1)]
IV_alpha_variables = LpVariable.matrix("alpha", alpha_variable_names, cat = "Binary", lowBound= 0 )
alpha_allocation = np.array(IV_alpha_variables).reshape(len(df_sheet_3),14+1)


#zeta variable m,t - enable to know the total amount of forage habitat period allowed by treatment 
zeta_variable_names = [str(i) for i in range (1, n_stands+1)]
IV_zeta_variables = LpVariable.matrix("zeta", zeta_variable_names, cat = "Binary", lowBound= 0 )
zeta_allocation = np.array(IV_zeta_variables).reshape(len(df_sheet_3))

#age
age_variable_names = [str(i) + ',' + str(j) for i in range (1, n_stands+1) for j in range (n_periods+1)]
IV_age_variables = LpVariable.matrix("age", age_variable_names, cat = "Integer", lowBound= 0 )
age_allocation = np.array(IV_age_variables).reshape(len(df_sheet_3),14+1)

In [ ]:
# Constraints 

## set the constraints 

## Constrain 1: Forage habitat 
### SUM (p+1, p+2) delta m,p' = 2*Xm,p + 2*Ym,p 
    ### add constraint if p > P 

## Constraint 2: One PCT and one harvest 
### SUM Xm,p <= 1 
### SUM Ym,p <= 1 with p >= 0 

## Constraint 3: PCT conditions 
### Xm,p * min_n_periods <= Age m,p  HERE min_n_periods = 3 means 15 years
### (40-Age m,p) + (1-Xm,p)*m >= 0 

## Constraint 4: Forage constraint, turn off 
### SUM delta m,p <= 2*Sum Xm,p + 2*Sum Ym,p + (1 + Zm - (Sum Xm,p + Sum Ym,p))M 
#### must add an indicator variable Z m,t = Sum Xm,p * Sum Ym,p 
    #### 2*Zm <= Sum Xm,p* Sum Ym,p 
    #### Sum Ym,p + Sum Xm,p - Zm <= 1 
    
## Age function 


## Constraints 5: Old growth - cover type 

#### Cover constraint 1 : ### 
### Age m,p - 24*omega m,p >= 0 
######  ==> forces omega to turn 0 if too young 

#### Cover constraint 2 : ### 
### m*omega m,p + 24 >= (1- Sum Ym,p) + Age m,p 
##### ==> if age is equal to or greater than 120 years omega would be forced to turn on 

## Constraint 6: Habitat availability 

## 1st ATTEMPT# |Sm|*alpha m,p - Sum omega k,p <= delta m,p (|Sm| - 1)
### alpha m,p <= delta m,t + omega m,p 
## 2nd ATTEMPT#
    #### 2*alpha m,p <= delta m,p + omega k,p for all k 
    #### delta m,p + omega k,p  - alpha m,p <= 1 for all k 
    ###### do not need since maximization problem!!!
## 3rd ATTEMPT# 
    #### alpha m,p <= (delta m,p + SUM omega k,p for all k)/|Sk|+1
    #### IF MAX then that is all 

## Constraint 7: turn ancient harvest and PCT treatment on 
## Yr and Xr_allocation = 1 
new_Y_p[0]

In [7]:
## Constrain 1: Forage habitat 
### SUM (p+1, p+2) delta m,p' = 2*Xm,p + 2*Ym,p 
### add constraint if p > P
## ==> ### SUM (p+1, p+2) delta m,p' <= 2*Xm,p + 2*Ym,p 
### add constraint if p > P

for m in range (n_stands):
    for p in range (n_periods):
        if p < n_periods-1 :
            model += delta_allocation[m][p+1] + delta_allocation[m][p+2] <= 2*X_allocation[m][p] + 2*Y_allocation[m][p], "Forhab_" + str(m+1) + "_" + str(p)
        elif p == n_periods-1:   
            model += delta_allocation[m][p+1] <= X_allocation[m][p] + Y_allocation[m][p], "Forhab_" + str(m+1) + "_" + str(p)


In [8]:
## Constraint 2: One PCT and one harvest 
### SUM Xm,p <= 1 
### SUM Ym,p <= 1 with p >= 0 

for m in range(n_stands): 
    model += lpSum(X_allocation[m][p] for p in range(1,n_periods+1))<=1, "One_PCT_" + str(m+1)
    model += lpSum(Y_allocation[m][p] for p in range(1,n_periods+1))<=1, "One_harv_" + str(m+1)

In [9]:
#Function Age :
for m in range(n_stands+1): 
    for t in range(n_periods): 
        for k in range(len(new_Y_p)): 
            if new_Y_m[k] == m:
                model+= (age_allocation[m-1][t] == 
                    t - new_Y_p[k] + lpSum(Y_allocation[m-1][p]*(new_Y_p[k]-p) for p in range(0, t)) 
                        , "age_" + str(m) + "_"+ str(t))


In [10]:
## Constraint 3: PCT conditions 
### Xm,p * 15/5 <= Age m,p 


for m in range(n_stands+1): 
    for k in range(len(new_Y_p)): 
        if new_Y_m[k] == m:
            for t in range(n_periods + 1):
                model +=(X_allocation[m-1][t]*min_yr/5 <= t - new_Y_p[k] + 
                         lpSum(Y_allocation[m-1][p]*(new_Y_p[k]-p) for p in range(0, t)) 
                      , "PCT15_"+ str(m)+ "_"+ str(t))
                

In [11]:
### (40-Age m,p) + (1-Xm,p)*M >= 0 

for m in range(n_stands+1): 
    for k in range(len(new_Y_p)): 
        if new_Y_m[k] == m:
            for t in range(n_periods + 1):
                model += (0 <= (1-X_allocation[m-1][t])*M + max_yr/5 - 
                          (t - new_Y_p[k] + lpSum(Y_allocation[m-1][p]*(new_Y_p[k]-p) for p in range(0, t))) 
                     , "PCT40_"+ str(m)+ "_"+ str(t))

In [12]:
## Constraint 4: Forage constraint, turn off 
### SUM delta m,t <= 2*Sum Xm,t + 2*Sum Ym,t + (1 + Sum Zm,t - (Sum Xm,t + Sum Ymt))M  

for m in range(n_stands+1):
    model += (lpSum(delta_allocation[m-1][p] for p in range(n_periods+1)) <= lpSum(2*X_allocation[m-1][p]
                                                                        + 2*Y_allocation[m-1][p]
                                                                        + (1 + zeta_allocation[m-1] #add zeta 
                                                                          - (X_allocation[m-1][p]
                                                                          + Y_allocation[m-1][p]))*M
                                                                        for p in range(n_periods+1)), 
              "Turnoff_forage_" + str(m))

In [13]:
## Constraint 4': switch zeta on or off 
#### 2*Zm <= Sum Xm,t + Sum Ym,t 
for m in range(n_stands+1):
    model += (lpSum(X_allocation[m-1][p] + Y_allocation[m-1][p] for p in range(n_periods+1)) >= 2*zeta_allocation[m-1], "zeta_con1_" + str(m)+ "_"+ str(p))      
#### Sum Ym,t + Sum Xm,t - Zm <= 1 
for m in range(n_stands+1):
    model += (lpSum(X_allocation[m-1][p] + Y_allocation[m-1][p] for p in range(n_periods+1)) - zeta_allocation[m-1] <= 1, "zeta_con2_" + str(m)+ "_"+ str(p))      

In [14]:
## Constraints 5: Old growth - cover type 
### omega m,p*24 <= Age m,p NOT WORKING
#### instead try: Age m,p - 24*(1-omega m,p) >= 24*omega NOT WORKING 
#### instead try: Age m,p - 24*omega m,p >= 0 

for m in range(n_stands+1): 
    for k in range(len(new_Y_p)): 
        if new_Y_m[k] == m:
            for t in range(n_periods + 1):
                model += (t - new_Y_p[k] + lpSum(Y_allocation[m-1][p]*(new_Y_p[k]-p) for p in range(0, t)) 
                        - (omega_allocation[m-1][t]*min_yr_cover/5) >= 0
                     , "Cover24_"+ str(m) + "_" +str(t))

                
### M*omega m,t + 24 >= (1- Sum Ym,t) + Age m,t          

In [15]:
### m*omega m,p + 24 >= (1- Sum Ym,p) + Age m,p 

for m in range(n_stands+1): 
    for k in range(len(new_Y_p)): 
        if new_Y_m[k] == m:
            for t in range(n_periods + 1):
                model+= (M*omega_allocation[m-1][t] + min_yr_cover/5 
                         >= 1-lpSum(Y_allocation[m-1][t]) + t - new_Y_p[k] 
                         + lpSum(Y_allocation[m-1][p]*(new_Y_p[k]-p) for p in range(0, t)) 
                         , "Cover_on_"+str(m)+ "_" +str(t))

In [16]:
## Constraint 6': 
#####alpha m,p <= delta m,p + SUM omega k,p for all k/|Sk|
for m in range(n_stands+1):
    for i in range(len(N_stands)):
        if m == N_stands[i]:
            a = []
            adj_ind = []
            if isinstance(Adj_stands["adj_i"][i], int):
                a = Adj_stands["adj_i"][i]
                adj_ind = i_Adj.index(a)                
                for p in range(n_periods + 1):
                     model += (alpha_allocation[m-1][p] <= (omega_allocation[adj_ind][p] 
                               + delta_allocation[m-1][p])/2
                      , "hab_av_" + str(m) + "_" + str(p))
            elif isinstance(Adj_stands["adj_i"][i], float):
                print(i, "_is_a_nan")
                for p in range(n_periods + 1):
                    model += alpha_allocation[m-1][p] == 0, "hab_av_" + str(m) + "_" + str(p)
                # shall we add a constraint there? 
            else:
                a = Adj_stands["adj_i"][i].split("_")
                a = [int(i) for i in a]
                for c in range(len(a)): 
                    adj_ind.append(i_Adj.index(a[c]))
                for p in range(n_periods + 1):
                    model += (alpha_allocation[m-1][p] 
                          <= (lpSum(omega_allocation[adj_ind[n]][p] for n in range(len(adj_ind)))
                             + delta_allocation[m-1][p])/(len(adj_ind)+1), "hab_av_" + str(m) + "_" + str(p))


59 _is_a_nan


In [17]:
#Constraint 7 : recorded PCT and harvest treatment = 1 

for m in range(len(new_Y_m)): 
    model += (Yr_allocation[m] == 1), "rec_harv_" + str(m)
    
for m in range(len(new_X_m)):
    model += (Xr_allocation[m] == 1), "rec_PCT" + str(m)

In [ ]:
#Constraint 8 : period requirements 
#for p in range(n_periods):
    #model+= (lpSum(alpha_allocation[m-1][p] for m in range(n_stands+1)) >= 2), "period_con_" + str(m) + "_" + str(p)

In [18]:
# Objective function 
obj_func = lpSum(alpha_allocation[i][j]*A_stands["area"][i] for i in range(91) for j in range(15))
model += obj_func              

In [19]:
#Save code into a lp file 
model.writeLP("model_DH_05_13_corr_avail.lp")


[X_1,0,
 X_1,1,
 X_1,10,
 X_1,11,
 X_1,12,
 X_1,13,
 X_1,14,
 X_1,2,
 X_1,3,
 X_1,4,
 X_1,5,
 X_1,6,
 X_1,7,
 X_1,8,
 X_1,9,
 X_1,_4,
 X_10,0,
 X_10,1,
 X_10,10,
 X_10,11,
 X_10,12,
 X_10,13,
 X_10,14,
 X_10,2,
 X_10,3,
 X_10,4,
 X_10,5,
 X_10,6,
 X_10,7,
 X_10,8,
 X_10,9,
 X_11,0,
 X_11,1,
 X_11,10,
 X_11,11,
 X_11,12,
 X_11,13,
 X_11,14,
 X_11,2,
 X_11,3,
 X_11,4,
 X_11,5,
 X_11,6,
 X_11,7,
 X_11,8,
 X_11,9,
 X_12,0,
 X_12,1,
 X_12,10,
 X_12,11,
 X_12,12,
 X_12,13,
 X_12,14,
 X_12,2,
 X_12,3,
 X_12,4,
 X_12,5,
 X_12,6,
 X_12,7,
 X_12,8,
 X_12,9,
 X_13,0,
 X_13,1,
 X_13,10,
 X_13,11,
 X_13,12,
 X_13,13,
 X_13,14,
 X_13,2,
 X_13,3,
 X_13,4,
 X_13,5,
 X_13,6,
 X_13,7,
 X_13,8,
 X_13,9,
 X_14,0,
 X_14,1,
 X_14,10,
 X_14,11,
 X_14,12,
 X_14,13,
 X_14,14,
 X_14,2,
 X_14,3,
 X_14,4,
 X_14,5,
 X_14,6,
 X_14,7,
 X_14,8,
 X_14,9,
 X_15,0,
 X_15,1,
 X_15,10,
 X_15,11,
 X_15,12,
 X_15,13,
 X_15,14,
 X_15,2,
 X_15,3,
 X_15,4,
 X_15,5,
 X_15,6,
 X_15,7,
 X_15,8,
 X_15,9,
 X_15,_4,
 X_16,0,
 X_16,1

In [ ]:
print(model)

In [20]:
#play around with CBC solver 
model.solve(PULP_CBC_CMD(msg=1))

status =  LpStatus[model.status]

print(status)

Optimal


In [21]:
print("Total area:", model.objective.value())

# Decision Variables

for v in model.variables():
    try:
        print(v.name,"=", v.value())
    except:
        print("error couldnt find value")
        
#Cplex.conflict.refine

Total area: 296.41237720460003
X_1,0 = 0.0
X_1,1 = 0.0
X_1,10 = 0.0
X_1,11 = 0.0
X_1,12 = 0.0
X_1,13 = 0.0
X_1,14 = 0.0
X_1,2 = 0.0
X_1,3 = 0.0
X_1,4 = 1.0
X_1,5 = 0.0
X_1,6 = 0.0
X_1,7 = 0.0
X_1,8 = 0.0
X_1,9 = 0.0
X_1,_4 = 1.0
X_10,0 = 0.0
X_10,1 = 0.0
X_10,10 = 0.0
X_10,11 = 0.0
X_10,12 = 0.0
X_10,13 = 0.0
X_10,14 = 0.0
X_10,2 = 0.0
X_10,3 = 1.0
X_10,4 = 0.0
X_10,5 = 0.0
X_10,6 = 0.0
X_10,7 = 0.0
X_10,8 = 0.0
X_10,9 = 0.0
X_11,0 = 0.0
X_11,1 = 0.0
X_11,10 = 0.0
X_11,11 = 0.0
X_11,12 = 0.0
X_11,13 = 0.0
X_11,14 = 1.0
X_11,2 = 0.0
X_11,3 = 0.0
X_11,4 = 0.0
X_11,5 = 0.0
X_11,6 = 0.0
X_11,7 = 0.0
X_11,8 = 0.0
X_11,9 = 0.0
X_12,0 = 0.0
X_12,1 = 0.0
X_12,10 = 0.0
X_12,11 = 0.0
X_12,12 = 0.0
X_12,13 = 0.0
X_12,14 = 0.0
X_12,2 = 0.0
X_12,3 = 0.0
X_12,4 = 0.0
X_12,5 = 0.0
X_12,6 = 0.0
X_12,7 = 0.0
X_12,8 = 0.0
X_12,9 = 0.0
X_13,0 = 0.0
X_13,1 = 0.0
X_13,10 = 0.0
X_13,11 = 0.0
X_13,12 = 0.0
X_13,13 = 0.0
X_13,14 = 0.0
X_13,2 = 0.0
X_13,3 = 0.0
X_13,4 = 0.0
X_13,5 = 0.0
X_13,6 = 0.0
X_13,7 = 0

X_57,0 = 0.0
X_57,1 = 0.0
X_57,10 = 0.0
X_57,11 = 0.0
X_57,12 = 0.0
X_57,13 = 0.0
X_57,14 = 0.0
X_57,2 = 0.0
X_57,3 = 0.0
X_57,4 = 0.0
X_57,5 = 0.0
X_57,6 = 0.0
X_57,7 = 0.0
X_57,8 = 0.0
X_57,9 = 0.0
X_58,0 = 0.0
X_58,1 = 0.0
X_58,10 = 0.0
X_58,11 = 0.0
X_58,12 = 0.0
X_58,13 = 0.0
X_58,14 = 0.0
X_58,2 = 0.0
X_58,3 = 0.0
X_58,4 = 0.0
X_58,5 = 0.0
X_58,6 = 0.0
X_58,7 = 0.0
X_58,8 = 0.0
X_58,9 = 0.0
X_59,0 = 0.0
X_59,1 = 0.0
X_59,10 = 0.0
X_59,11 = 0.0
X_59,12 = 0.0
X_59,13 = 0.0
X_59,14 = 0.0
X_59,2 = 0.0
X_59,3 = 1.0
X_59,4 = 0.0
X_59,5 = 0.0
X_59,6 = 0.0
X_59,7 = 0.0
X_59,8 = 0.0
X_59,9 = 0.0
X_6,0 = 0.0
X_6,1 = 0.0
X_6,10 = 0.0
X_6,11 = 0.0
X_6,12 = 0.0
X_6,13 = 0.0
X_6,14 = 1.0
X_6,2 = 0.0
X_6,3 = 0.0
X_6,4 = 0.0
X_6,5 = 0.0
X_6,6 = 0.0
X_6,7 = 0.0
X_6,8 = 0.0
X_6,9 = 0.0
X_60,0 = 0.0
X_60,1 = 0.0
X_60,10 = 0.0
X_60,11 = 0.0
X_60,12 = 0.0
X_60,13 = 0.0
X_60,14 = 0.0
X_60,2 = 0.0
X_60,3 = 0.0
X_60,4 = 0.0
X_60,5 = 0.0
X_60,6 = 0.0
X_60,7 = 0.0
X_60,8 = 0.0
X_60,9 = 0.0
X_61,0 = 0.0
X_

Y_14,1 = 0.0
Y_14,10 = 0.0
Y_14,11 = 0.0
Y_14,12 = 0.0
Y_14,13 = 0.0
Y_14,14 = 0.0
Y_14,2 = 0.0
Y_14,3 = 0.0
Y_14,4 = 0.0
Y_14,5 = 0.0
Y_14,6 = 0.0
Y_14,7 = 0.0
Y_14,8 = 0.0
Y_14,9 = 0.0
Y_14,_26 = 1.0
Y_15,0 = 1.0
Y_15,1 = 0.0
Y_15,10 = 0.0
Y_15,11 = 0.0
Y_15,12 = 0.0
Y_15,13 = 0.0
Y_15,14 = 0.0
Y_15,2 = 0.0
Y_15,3 = 0.0
Y_15,4 = 0.0
Y_15,5 = 0.0
Y_15,6 = 0.0
Y_15,7 = 0.0
Y_15,8 = 0.0
Y_15,9 = 0.0
Y_15,_10 = 1.0
Y_16,0 = 0.0
Y_16,1 = 0.0
Y_16,10 = 0.0
Y_16,11 = 0.0
Y_16,12 = 0.0
Y_16,13 = 0.0
Y_16,14 = 1.0
Y_16,2 = 0.0
Y_16,3 = 0.0
Y_16,4 = 0.0
Y_16,5 = 0.0
Y_16,6 = 0.0
Y_16,7 = 0.0
Y_16,8 = 0.0
Y_16,9 = 0.0
Y_16,_12 = 1.0
Y_17,0 = 1.0
Y_17,1 = 0.0
Y_17,10 = 0.0
Y_17,11 = 0.0
Y_17,12 = 0.0
Y_17,13 = 0.0
Y_17,14 = 0.0
Y_17,2 = 0.0
Y_17,3 = 0.0
Y_17,4 = 0.0
Y_17,5 = 0.0
Y_17,6 = 0.0
Y_17,7 = 0.0
Y_17,8 = 0.0
Y_17,9 = 0.0
Y_17,_12 = 1.0
Y_18,0 = 1.0
Y_18,1 = 0.0
Y_18,10 = 0.0
Y_18,11 = 0.0
Y_18,12 = 0.0
Y_18,13 = 0.0
Y_18,14 = 0.0
Y_18,2 = 0.0
Y_18,3 = 0.0
Y_18,4 = 0.0
Y_18,5 = 0.0
Y_18,

Y_5,11 = 0.0
Y_5,12 = 0.0
Y_5,13 = 0.0
Y_5,14 = 1.0
Y_5,2 = 0.0
Y_5,3 = 0.0
Y_5,4 = 0.0
Y_5,5 = 0.0
Y_5,6 = 0.0
Y_5,7 = 0.0
Y_5,8 = 0.0
Y_5,9 = 0.0
Y_5,_26 = 1.0
Y_50,0 = 0.0
Y_50,1 = 1.0
Y_50,10 = 0.0
Y_50,11 = 0.0
Y_50,12 = 0.0
Y_50,13 = 0.0
Y_50,14 = 0.0
Y_50,2 = 0.0
Y_50,3 = 0.0
Y_50,4 = 0.0
Y_50,5 = 0.0
Y_50,6 = 0.0
Y_50,7 = 0.0
Y_50,8 = 0.0
Y_50,9 = 0.0
Y_50,_7 = 1.0
Y_51,0 = 1.0
Y_51,1 = 0.0
Y_51,10 = 0.0
Y_51,11 = 0.0
Y_51,12 = 0.0
Y_51,13 = 0.0
Y_51,14 = 0.0
Y_51,2 = 0.0
Y_51,3 = 0.0
Y_51,4 = 0.0
Y_51,5 = 0.0
Y_51,6 = 0.0
Y_51,7 = 0.0
Y_51,8 = 0.0
Y_51,9 = 0.0
Y_51,_12 = 1.0
Y_52,0 = 0.0
Y_52,1 = 1.0
Y_52,10 = 0.0
Y_52,11 = 0.0
Y_52,12 = 0.0
Y_52,13 = 0.0
Y_52,14 = 0.0
Y_52,2 = 0.0
Y_52,3 = 0.0
Y_52,4 = 0.0
Y_52,5 = 0.0
Y_52,6 = 0.0
Y_52,7 = 0.0
Y_52,8 = 0.0
Y_52,9 = 0.0
Y_52,_26 = 1.0
Y_53,0 = 1.0
Y_53,1 = 0.0
Y_53,10 = 0.0
Y_53,11 = 0.0
Y_53,12 = 0.0
Y_53,13 = 0.0
Y_53,14 = 1.0
Y_53,2 = 0.0
Y_53,3 = 0.0
Y_53,4 = 0.0
Y_53,5 = 0.0
Y_53,6 = 0.0
Y_53,7 = 0.0
Y_53,8 = 0.0
Y_53,9 

age_17,9 = 9.0
age_18,0 = 12.0
age_18,1 = 1.0
age_18,10 = 10.0
age_18,11 = 11.0
age_18,12 = 12.0
age_18,13 = 13.0
age_18,2 = 2.0
age_18,3 = 3.0
age_18,4 = 4.0
age_18,5 = 5.0
age_18,6 = 6.0
age_18,7 = 7.0
age_18,8 = 8.0
age_18,9 = 9.0
age_19,0 = 7.0
age_19,1 = 8.0
age_19,10 = 2.0
age_19,11 = 3.0
age_19,12 = 4.0
age_19,13 = 5.0
age_19,2 = 9.0
age_19,3 = 10.0
age_19,4 = 11.0
age_19,5 = 12.0
age_19,6 = 13.0
age_19,7 = 14.0
age_19,8 = 15.0
age_19,9 = 1.0
age_2,0 = 7.0
age_2,1 = 8.0
age_2,10 = 5.0
age_2,11 = 6.0
age_2,12 = 7.0
age_2,13 = 8.0
age_2,2 = 9.0
age_2,3 = 10.0
age_2,4 = 11.0
age_2,5 = 12.0
age_2,6 = 1.0
age_2,7 = 2.0
age_2,8 = 3.0
age_2,9 = 4.0
age_20,0 = 7.0
age_20,1 = 8.0
age_20,10 = 17.0
age_20,11 = 18.0
age_20,12 = 19.0
age_20,13 = 20.0
age_20,2 = 9.0
age_20,3 = 10.0
age_20,4 = 11.0
age_20,5 = 12.0
age_20,6 = 13.0
age_20,7 = 14.0
age_20,8 = 15.0
age_20,9 = 16.0
age_21,0 = 10.0
age_21,1 = 11.0
age_21,10 = 20.0
age_21,11 = 21.0
age_21,12 = 22.0
age_21,13 = 23.0
age_21,2 = 12.0
ag

...

alpha_77,12 = 0.0
alpha_77,13 = 0.0
alpha_77,14 = 0.0
alpha_77,2 = 0.0
alpha_77,3 = 0.0
alpha_77,4 = 0.0
alpha_77,5 = 0.0
alpha_77,6 = 0.0
alpha_77,7 = 0.0
alpha_77,8 = 0.0
alpha_77,9 = 0.0
alpha_78,0 = 0.0
alpha_78,1 = 0.0
alpha_78,10 = 0.0
alpha_78,11 = 0.0
alpha_78,12 = 0.0
alpha_78,13 = 0.0
alpha_78,14 = 0.0
alpha_78,2 = 0.0
alpha_78,3 = 0.0
alpha_78,4 = 0.0
alpha_78,5 = 0.0
alpha_78,6 = 0.0
alpha_78,7 = 0.0
alpha_78,8 = 0.0
alpha_78,9 = 0.0
alpha_79,0 = 0.0
alpha_79,1 = 0.0
alpha_79,10 = 0.0
alpha_79,11 = 0.0
alpha_79,12 = 0.0
alpha_79,13 = 0.0
alpha_79,14 = 0.0
alpha_79,2 = 0.0
alpha_79,3 = 0.0
alpha_79,4 = 0.0
alpha_79,5 = 0.0
alpha_79,6 = 0.0
alpha_79,7 = 0.0
alpha_79,8 = 0.0
alpha_79,9 = 0.0
alpha_8,0 = 0.0
alpha_8,1 = 0.0
alpha_8,10 = 0.0
alpha_8,11 = 0.0
alpha_8,12 = 0.0
alpha_8,13 = 0.0
alpha_8,14 = 0.0
alpha_8,2 = 0.0
alpha_8,3 = 0.0
alpha_8,4 = 0.0
alpha_8,5 = 0.0
alpha_8,6 = 0.0
alpha_8,7 = 0.0
alpha_8,8 = 0.0
alpha_8,9 = 0.0
alpha_80,0 = 0.0
alpha_80,1 = 0.0
alpha_80,10

delta_51,2 = 0.0
delta_51,3 = 0.0
delta_51,4 = 0.0
delta_51,5 = 0.0
delta_51,6 = 0.0
delta_51,7 = 0.0
delta_51,8 = 0.0
delta_51,9 = 0.0
delta_52,0 = 0.0
delta_52,1 = 0.0
delta_52,10 = 0.0
delta_52,11 = 0.0
delta_52,12 = 0.0
delta_52,13 = 0.0
delta_52,14 = 0.0
delta_52,2 = 0.0
delta_52,3 = 0.0
delta_52,4 = 0.0
delta_52,5 = 0.0
delta_52,6 = 0.0
delta_52,7 = 0.0
delta_52,8 = 0.0
delta_52,9 = 0.0
delta_53,0 = 0.0
delta_53,1 = 0.0
delta_53,10 = 0.0
delta_53,11 = 0.0
delta_53,12 = 0.0
delta_53,13 = 0.0
delta_53,14 = 0.0
delta_53,2 = 0.0
delta_53,3 = 0.0
delta_53,4 = 0.0
delta_53,5 = 0.0
delta_53,6 = 0.0
delta_53,7 = 0.0
delta_53,8 = 0.0
delta_53,9 = 0.0
delta_54,0 = 0.0
delta_54,1 = 0.0
delta_54,10 = 0.0
delta_54,11 = 0.0
delta_54,12 = 0.0
delta_54,13 = 0.0
delta_54,14 = 0.0
delta_54,2 = 0.0
delta_54,3 = 0.0
delta_54,4 = 0.0
delta_54,5 = 0.0
delta_54,6 = 0.0
delta_54,7 = 0.0
delta_54,8 = 0.0
delta_54,9 = 0.0
delta_55,0 = 0.0
delta_55,1 = 0.0
delta_55,10 = 0.0
delta_55,11 = 0.0
delta_55,12 = 

delta_83,3 = 0.0
delta_83,4 = 0.0
delta_83,5 = 0.0
delta_83,6 = 0.0
delta_83,7 = 0.0
delta_83,8 = 0.0
delta_83,9 = 0.0
delta_84,0 = 0.0
delta_84,1 = 0.0
delta_84,10 = 0.0
delta_84,11 = 0.0
delta_84,12 = 0.0
delta_84,13 = 0.0
delta_84,14 = 0.0
delta_84,2 = 0.0
delta_84,3 = 0.0
delta_84,4 = 0.0
delta_84,5 = 0.0
delta_84,6 = 0.0
delta_84,7 = 0.0
delta_84,8 = 0.0
delta_84,9 = 0.0
delta_85,0 = 0.0
delta_85,1 = 0.0
delta_85,10 = 0.0
delta_85,11 = 0.0
delta_85,12 = 0.0
delta_85,13 = 0.0
delta_85,14 = 0.0
delta_85,2 = 0.0
delta_85,3 = 0.0
delta_85,4 = 0.0
delta_85,5 = 0.0
delta_85,6 = 0.0
delta_85,7 = 0.0
delta_85,8 = 0.0
delta_85,9 = 0.0
delta_86,0 = 0.0
delta_86,1 = 0.0
delta_86,10 = 0.0
delta_86,11 = 0.0
delta_86,12 = 0.0
delta_86,13 = 0.0
delta_86,14 = 0.0
delta_86,2 = 0.0
delta_86,3 = 0.0
delta_86,4 = 0.0
delta_86,5 = 0.0
delta_86,6 = 0.0
delta_86,7 = 0.0
delta_86,8 = 0.0
delta_86,9 = 0.0
delta_87,0 = 0.0
delta_87,1 = 0.0
delta_87,10 = 0.0
delta_87,11 = 0.0
delta_87,12 = 0.0
delta_87,13 =

omega_33,14 = 0.0
omega_33,2 = 1.0
omega_33,3 = 1.0
omega_33,4 = 1.0
omega_33,5 = 1.0
omega_33,6 = 1.0
omega_33,7 = 1.0
omega_33,8 = 1.0
omega_33,9 = 1.0
omega_34,0 = 0.0
omega_34,1 = 0.0
omega_34,10 = 0.0
omega_34,11 = 0.0
omega_34,12 = 0.0
omega_34,13 = 0.0
omega_34,14 = 0.0
omega_34,2 = 0.0
omega_34,3 = 0.0
omega_34,4 = 0.0
omega_34,5 = 0.0
omega_34,6 = 0.0
omega_34,7 = 0.0
omega_34,8 = 0.0
omega_34,9 = 0.0
omega_35,0 = 0.0
omega_35,1 = 0.0
omega_35,10 = 0.0
omega_35,11 = 0.0
omega_35,12 = 0.0
omega_35,13 = 0.0
omega_35,14 = 0.0
omega_35,2 = 0.0
omega_35,3 = 0.0
omega_35,4 = 0.0
omega_35,5 = 0.0
omega_35,6 = 0.0
omega_35,7 = 0.0
omega_35,8 = 0.0
omega_35,9 = 0.0
omega_36,0 = 0.0
omega_36,1 = 0.0
omega_36,10 = 0.0
omega_36,11 = 0.0
omega_36,12 = 0.0
omega_36,13 = 0.0
omega_36,14 = 0.0
omega_36,2 = 0.0
omega_36,3 = 0.0
omega_36,4 = 0.0
omega_36,5 = 0.0
omega_36,6 = 0.0
omega_36,7 = 0.0
omega_36,8 = 0.0
omega_36,9 = 0.0
omega_37,0 = 0.0
omega_37,1 = 0.0
omega_37,10 = 0.0
omega_37,11 = 

zeta_12 = 0.0
zeta_13 = 0.0
zeta_14 = 0.0
zeta_15 = 1.0
zeta_16 = 0.0
zeta_17 = 1.0
zeta_18 = 1.0
zeta_19 = 0.0
zeta_2 = 1.0
zeta_20 = 0.0
zeta_21 = 0.0
zeta_22 = 1.0
zeta_23 = 1.0
zeta_24 = 0.0
zeta_25 = 0.0
zeta_26 = 0.0
zeta_27 = 0.0
zeta_28 = 0.0
zeta_29 = 1.0
zeta_3 = 0.0
zeta_30 = 0.0
zeta_31 = 1.0
zeta_32 = 1.0
zeta_33 = 1.0
zeta_34 = 1.0
zeta_35 = 0.0
zeta_36 = 1.0
zeta_37 = 0.0
zeta_38 = 1.0
zeta_39 = 0.0
zeta_4 = 0.0
zeta_40 = 1.0
zeta_41 = 0.0
zeta_42 = 1.0
zeta_43 = 1.0
zeta_44 = 0.0
zeta_45 = 1.0
zeta_46 = 0.0
zeta_47 = 0.0
zeta_48 = 1.0
zeta_49 = 0.0
zeta_5 = 0.0
zeta_50 = 1.0
zeta_51 = 1.0
zeta_52 = 1.0
zeta_53 = 1.0
zeta_54 = 1.0
zeta_55 = 1.0
zeta_56 = 1.0
zeta_57 = 0.0
zeta_58 = 0.0
zeta_59 = 1.0
zeta_6 = 1.0
zeta_60 = 0.0
zeta_61 = 1.0
zeta_62 = 1.0
zeta_63 = 0.0
zeta_64 = 1.0
zeta_65 = 0.0
zeta_66 = 0.0
zeta_67 = 1.0
zeta_68 = 1.0
zeta_69 = 1.0
zeta_7 = 1.0
zeta_70 = 1.0
zeta_71 = 1.0
zeta_72 = 1.0
zeta_73 = 0.0
zeta_74 = 0.0
zeta_75 = 1.0
zeta_76 = 0.0
zeta_77 = 1.